# SQL

In [1]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

Found pyproject.toml from '/Users/adam/@/jupyter-codespace'

Use `%%` for multi-line statements.

In [2]:
%%sql
DROP TABLE IF EXISTS languages;
CREATE TABLE languages (
  name VARCHAR,
  rating REAL,
  change REAL
);
INSERT INTO languages VALUES ('Python', 14.44, 2.48);
INSERT INTO languages VALUES ('C', 13.13, 1.50);
INSERT INTO languages VALUES ('Java', 11.59, 0.40);
INSERT INTO languages VALUES ('C++', 10.00, 1.98);

,Success


In [3]:
%%sql
SELECT * FROM languages;

,name,rating,change
0,Python,14.44,2.48
1,C,13.13,1.50
2,Java,11.59,0.40
3,C++,10.00,1.98
